## Import and Twitter Auth

In [1]:
# Module imports 
import tweepy
import time
import pandas as pd
import numpy as np
import json
import time
from random import randint, random
from googlesearch import search as gsearch
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pathlib
import os
from dotenv import load_dotenv

# name the file we want
env_file_name = 'lrhf97_twit_cred.env'
parent_dir = pathlib.Path.cwd().parent

env_path = parent_dir.joinpath(env_file_name)

load_dotenv(env_path)
test_cred = os.getenv("test_cred")

api_key = os.getenv("API_KEY")
secret_key = os.getenv("API_SECRET")
access_token = ''
secret_token = ''

def authenticate(api_key, secret_key, access_token, secret_token):

    auth = tweepy.OAuthHandler(api_key, secret_key)
    auth.set_access_token(access_token, secret_token)
    api = tweepy.API(auth, 
                     wait_on_rate_limit=True, 
                     wait_on_rate_limit_notify=True)
  
    return api
    
api = authenticate(api_key, secret_key, access_token, secret_token)

## Twitter Search term selction

In [3]:
text_query = '2020 US Election'
count = 15
try:
 # Creation of query method using parameters
    tweets = tweepy.Cursor(api.search,q=text_query).items(count)
 
 # Pulling information from tweets iterable object
    tweets_list = [[tweet.created_at, tweet.id, tweet.text] for tweet in tweets]
 
 # Creation of dataframe from tweets list
 # Add or remove columns as you remove tweet information
    tweetsQ_df = pd.DataFrame(tweets_list)
 
except BaseException as e:
    print('failed on_status,',str(e))
    time.sleep(3)

### Extract Coordinates and Place

In [2]:
# Function created to extract coordinates from tweet if it has coordinate info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_coordinates(row):
    if row['Tweet Coordinates']:
        return row['Tweet Coordinates']['coordinates']
    else:
        return None
        
# Function created to extract place such as city, state or country from tweet if it has place info
# Tweets tend to have null so important to run check
# Make sure to run this cell as it is used in a lot of different functions below
def extract_place(row):
    if row['Place Info']:
        return row['Place Info'].full_name
    else:
        return None

In [4]:
text_query = '@RidgeSupply'
count = 15

tweets = tweepy.Cursor(api.search, q=text_query,tweet_mode='extended').items(count)

# Pulling information from tweets iterable 
tweets_list = [[tweet.full_text, tweet.created_at, tweet.id_str, tweet.user.screen_name,tweet.user.location, tweet.coordinates, tweet.place, tweet.retweet_count, tweet.favorite_count, tweet.lang, tweet.source, tweet.in_reply_to_status_id_str, tweet.in_reply_to_user_id_str, tweet.is_quote_status] for tweet in tweets]

# Creation of dataframe from tweets list
tweetsINFO_df = pd.DataFrame(tweets_list,columns=['Tweet Text', 'Tweet Datetime', 'Tweet Id', 'Twitter @ Name','User Home Base', 'Tweet Coordinates', 'Place Info', 'Retweets', 'Favorites', 'Language', 'Source', 'Replied Tweet Id', 'Replied Tweet User Id Str', 'Quote Status Bool'])

# Checks if there are coordinates attached to tweets, if so extracts them
tweetsINFO_df['Tweet Coordinates'] = tweetsINFO_df.apply(extract_coordinates,axis=1)
 
# Checks if there is place information available, if so extracts them
tweetsINFO_df['Place Info'] = tweetsINFO_df.apply(extract_place,axis=1)

In [5]:
tweetsINFO_df

,Tweet Text,Tweet Datetime,Tweet Id,Twitter @ Name,User Home Base,Tweet Coordinates,Place Info,Retweets,Favorites,Language,Source,Replied Tweet Id,Replied Tweet User Id Str,Quote Status Bool
0,"Another Day, Another Ride. Hmm, which socks sh...",2021-09-16 18:00:47,1438563532056969216,DumbMoney_AMC,,None,None,0,0,en,Twitter for Android,None,None,False
1,RT @leroys_dog: Dear Matt @ridgesupply -\n\nle...,2021-09-15 15:51:18,1438168557691473924,joand315,Chicago,None,None,1,0,en,Twitter for Android,None,None,False
2,Dear Matt @ridgesupply -\n\nleroy figured your...,2021-09-15 14:00:38,1438140708070830080,leroys_dog,"Brooklyn, NY",None,None,1,1,en,Twitter for iPhone,None,None,False


In [17]:
tweetsINFO_df['Tweet Text'][0]

'RT @SaltyMacTavish: Thinking of buying a new 24’ fiberglass extension ladder but I also have oak I could rive into rounds and sassafras I c…'